In [ ]:
%run 'common-functions.ipynb'

In [ ]:
CLOUDWATCH_STARTSWITH_FILTER = "Ds"


In [ ]:
@dask.delayed
def get_log_groups(client, account_id, account_region):
    paginator = client.get_paginator("describe_log_groups")
    page_iterator = paginator.paginate()
    data = []
    data = [page["logGroups"] for page in page_iterator]
    df = pd.DataFrame([item for sublist in data for item in sublist])
    df["account_id"] = account_id
    df["region"] = account_region
    return df


In [ ]:
# fetch accounts from metadata once - so dont need to keep taxing ddb in case of issue
accounts = fetch_accounts_from_metadata()


In [ ]:
%%time
# Fetch all cloudwatch log groups across all spokes that dont have a retentionInDays value set
master_df = pd.DataFrame()
dfs = []
print(f"Accounts to process: {len(accounts)}")
for account in accounts:
    account_id = account.get("account")
    account_region = account.get("region")
    if account_region:
        client = assume_return_boto_client("logs", account_id, account_region)
        dfs.append(get_log_groups(client, account_id, account_region))

graph = dask.delayed()(dfs)
results = graph.compute()
master_df = pd.concat(results)

In [ ]:
# show data frame
master_df


In [ ]:
filtered = master_df[
    master_df["logGroupName"].str.startswith(CLOUDWATCH_STARTSWITH_FILTER)
]
filtered
